In [1]:
import numpy as np
from sklearn import preprocessing, neighbors
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix


import pandas as pd

In [8]:
def nestedCV_pcutoffrange(receptor_num):
    # disable chained assignments
    pd.options.mode.chained_assignment = None 
    
    receptor_names = ['5HT2B','A2A','Beta 2','H1','M1','OPRD','OPRK','OPRM']
    holdout_receptor = receptor_names[receptor_num]
    del receptor_names[receptor_num]
    
    max_PPVS = []
    p_index_list = []
   
    for receptor_name in receptor_names:
        df = pd.read_csv('data\_All_Receptors_runs_1_2_3_binary.csv')
        df.drop(['Active_Rate','Enrichment', 'GH', 'Actives', 'filename', 'fbase', 'hyd', 'don', 'acc', 'ani', 'cat', 'aro', 'donhyd', 'catdon', 'hydaro', 'aniacc', 'donacc','don_prop', 'acc_prop', 'ani_prop', 'cat_prop', 'aro_prop', 'donhyd_prop', 'hydaro_prop', 'donacc_prop', 'min_feat', 'all_same'], 1, inplace=True)
        df.fillna(-99999)
        testset = df.loc[df['receptor'] == receptor_name]
        df = df[df['receptor'] != holdout_receptor]
        trainset = df[df['receptor'] != receptor_name]
        testset.drop('receptor',1, inplace=True)
        trainset.drop('receptor',1, inplace=True)
        
        if receptor_name == '5HT2B':
             print('Predictors: ', list(trainset.columns.values))
       
        
        #make instance of model
        # all parameters not specified are set to their defaults
        logisticRegr = LogisticRegression(max_iter = 5000)
        
        #split features and labels
        x_train = np.array(trainset.drop(['quality'], 1))
        y_train = np.array(trainset['quality'])
        
        x_test = np.array(testset.drop(['quality'], 1))
        y_test = np.array(testset['quality'])
        
        logisticRegr.fit(x_train, y_train)
        
        #predictions = logisticRegr.predict(x_test)
        
        print('receptor','\t','p_cutoff','\t','PPV')
        print('-----------------------------------')
        
        #threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
        threshold_list = np.arange(0.0,1.0,0.01)
        PPV_values = []
        
        for i in threshold_list:
            y_pred = (logisticRegr.predict_proba(x_test)[:,1] >= i).astype(bool) # set threshold as i
            #confusion_matrix, upper row is not quality, lower row is quality. left column is classified
            #as NQ, right column is classified as Q
            confmat = confusion_matrix(y_test, y_pred, labels=[0,1]).ravel()
            #print(confmat)
            #print(confmat[1])
            FP = (confmat[1])
            #print(confmat[3])
            TP = (confmat[3])
            if FP != 0 and TP != 0:
                PPV = (TP / (TP + FP))
                PPV_values.append(PPV)
                print(receptor_name,'\t', i, '\t', round(PPV,2))
            else:
                PPV = 0.00
                PPV_values.append(PPV)
                print(receptor_name,'\t', i, '\t', round(PPV,2))
        
        
        #print(PPV_values)
        max_PPV_value = max(PPV_values)
        max_PPVS.append(max_PPV_value)
        p_index_list.append(PPV_values.index(max_PPV_value))
        #print(max_PPVS)
        #print(p_index_list)
    
    print('\n')
    print('receptor','\t','p_cutoff','\t','PPV')
    print('-----------------------------------')
    
    i=0;
    for receptor_name in receptor_names:
        print(receptor_name,'\t', threshold_list[p_index_list[i]], '\t', round(max_PPVS[i],2))
        i+=1
        
        
        # Use score method to get accuracy of model
        #score = logisticRegr.score(x_test, y_test)
        #print(score)
        
        

In [21]:
def ph4_classify(receptor_num, p_cutoff):
    receptor_names = ['5HT2B','A2A','Beta 2','H1','M1','OPRD','OPRK','OPRM']
    receptor_name = receptor_names[receptor_num]
    
    df = pd.read_csv('data\_All_Receptors_runs_1_2_3_binary.csv')
    df.drop(['Active_Rate','Enrichment', 'GH', 'Actives', 'filename', 'fbase', 'hyd', 'don', 'acc', 'ani', 'cat', 'aro', 'donhyd', 'catdon', 'hydaro', 'aniacc', 'donacc','don_prop', 'acc_prop', 'ani_prop', 'cat_prop', 'aro_prop', 'donhyd_prop', 'hydaro_prop', 'donacc_prop','min_feat', 'all_same'], 1, inplace=True)
    df.fillna(-99999)
    testset = df.loc[df['receptor'] == receptor_name]
    trainset = df[df['receptor'] != receptor_name]
    testset.drop('receptor',1, inplace=True)
    trainset.drop('receptor',1, inplace=True)
    
    #make instance of model
    # all parameters not specified are set to their defaults
    logisticRegr = LogisticRegression(max_iter = 5000)
        
    #split features and labels
    x_train = np.array(trainset.drop(['quality'], 1))
    y_train = np.array(trainset['quality'])
        
    x_test = np.array(testset.drop(['quality'], 1))
    y_test = np.array(testset['quality'])
        
    logisticRegr.fit(x_train, y_train)
    
    y_pred = (logisticRegr.predict_proba(x_test)[:,1] >= p_cutoff).astype(bool) # set threshold as p_cutoff
    #confusion_matrix, upper row is not quality, lower row is quality. left column is classified
    #as NQ, right column is classified as Q
    cm = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)
    
    confmat = confusion_matrix(y_test, y_pred, labels=[0,1]).ravel()
    FP = (confmat[1])
    TP = (confmat[3])

    PPV = (TP / (TP + FP))
    
    print(cm,'\n')

    print('PPV:', format(PPV, '.2f'))
    

In [9]:
nestedCV_pcutoffrange(0)

receptor 	 p_cutoff 	 PPV
-----------------------------------
A2A 	 0.0 	 0.01
A2A 	 0.01 	 0.03
A2A 	 0.02 	 0.04
A2A 	 0.03 	 0.05
A2A 	 0.04 	 0.05
A2A 	 0.05 	 0.06
A2A 	 0.06 	 0.07
A2A 	 0.07 	 0.07
A2A 	 0.08 	 0.08
A2A 	 0.09 	 0.08
A2A 	 0.1 	 0.08
A2A 	 0.11 	 0.09
A2A 	 0.12 	 0.1
A2A 	 0.13 	 0.1
A2A 	 0.14 	 0.1
A2A 	 0.15 	 0.11
A2A 	 0.16 	 0.11
A2A 	 0.17 	 0.11
A2A 	 0.18 	 0.11
A2A 	 0.19 	 0.12
A2A 	 0.2 	 0.12
A2A 	 0.21 	 0.09
A2A 	 0.22 	 0.07
A2A 	 0.23 	 0.06
A2A 	 0.24 	 0.07
A2A 	 0.25 	 0.07
A2A 	 0.26 	 0.06
A2A 	 0.27 	 0.05
A2A 	 0.28 	 0.02
A2A 	 0.29 	 0.02
A2A 	 0.3 	 0.02
A2A 	 0.31 	 0.03
A2A 	 0.32 	 0.03
A2A 	 0.33 	 0.04
A2A 	 0.34 	 0.04
A2A 	 0.35000000000000003 	 0.04
A2A 	 0.36 	 0.05
A2A 	 0.37 	 0.05
A2A 	 0.38 	 0.05
A2A 	 0.39 	 0.0
A2A 	 0.4 	 0.0
A2A 	 0.41000000000000003 	 0.0
A2A 	 0.42 	 0.0
A2A 	 0.43 	 0.0
A2A 	 0.44 	 0.0
A2A 	 0.45 	 0.0
A2A 	 0.46 	 0.0
A2A 	 0.47000000000000003 	 0.0
A2A 	 0.48 	 0.0
A2A 	 0.49 	 0.0
A2A 	 0.5 	 

OPRD 	 0.17 	 0.3
OPRD 	 0.18 	 0.31
OPRD 	 0.19 	 0.31
OPRD 	 0.2 	 0.34
OPRD 	 0.21 	 0.37
OPRD 	 0.22 	 0.4
OPRD 	 0.23 	 0.4
OPRD 	 0.24 	 0.44
OPRD 	 0.25 	 0.49
OPRD 	 0.26 	 0.52
OPRD 	 0.27 	 0.56
OPRD 	 0.28 	 0.6
OPRD 	 0.29 	 0.63
OPRD 	 0.3 	 0.62
OPRD 	 0.31 	 0.62
OPRD 	 0.32 	 0.69
OPRD 	 0.33 	 0.71
OPRD 	 0.34 	 0.73
OPRD 	 0.35000000000000003 	 0.75
OPRD 	 0.36 	 0.82
OPRD 	 0.37 	 0.89
OPRD 	 0.38 	 0.88
OPRD 	 0.39 	 0.93
OPRD 	 0.4 	 0.93
OPRD 	 0.41000000000000003 	 0.93
OPRD 	 0.42 	 0.92
OPRD 	 0.43 	 0.92
OPRD 	 0.44 	 0.92
OPRD 	 0.45 	 0.91
OPRD 	 0.46 	 0.89
OPRD 	 0.47000000000000003 	 0.89
OPRD 	 0.48 	 0.88
OPRD 	 0.49 	 0.83
OPRD 	 0.5 	 0.8
OPRD 	 0.51 	 0.75
OPRD 	 0.52 	 0.67
OPRD 	 0.53 	 0.5
OPRD 	 0.54 	 0.0
OPRD 	 0.55 	 0.0
OPRD 	 0.56 	 0.0
OPRD 	 0.5700000000000001 	 0.0
OPRD 	 0.58 	 0.0
OPRD 	 0.59 	 0.0
OPRD 	 0.6 	 0.0
OPRD 	 0.61 	 0.0
OPRD 	 0.62 	 0.0
OPRD 	 0.63 	 0.0
OPRD 	 0.64 	 0.0
OPRD 	 0.65 	 0.0
OPRD 	 0.66 	 0.0
OPRD 	 0.67 	 0

In [10]:
nestedCV_pcutoffrange(1)

Predictors:  ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 's_score', 'hyd_prop', 'catdon_prop', 'aniacc_prop', 'quality']
receptor 	 p_cutoff 	 PPV
-----------------------------------
5HT2B 	 0.0 	 0.03
5HT2B 	 0.01 	 0.06
5HT2B 	 0.02 	 0.07
5HT2B 	 0.03 	 0.08
5HT2B 	 0.04 	 0.08
5HT2B 	 0.05 	 0.09
5HT2B 	 0.06 	 0.09
5HT2B 	 0.07 	 0.09
5HT2B 	 0.08 	 0.09
5HT2B 	 0.09 	 0.08
5HT2B 	 0.1 	 0.08
5HT2B 	 0.11 	 0.08
5HT2B 	 0.12 	 0.06
5HT2B 	 0.13 	 0.06
5HT2B 	 0.14 	 0.06
5HT2B 	 0.15 	 0.06
5HT2B 	 0.16 	 0.05
5HT2B 	 0.17 	 0.04
5HT2B 	 0.18 	 0.04
5HT2B 	 0.19 	 0.03
5HT2B 	 0.2 	 0.03
5HT2B 	 0.21 	 0.03
5HT2B 	 0.22 	 0.03
5HT2B 	 0.23 	 0.03
5HT2B 	 0.24 	 0.03
5HT2B 	 0.25 	 0.03
5HT2B 	 0.26 	 0.02
5HT2B 	 0.27 	 0.02
5HT2B 	 0.28 	 0.01
5HT2B 	 0.29 	 0.02
5HT2B 	 0.3 	 0.01
5HT2B 	 0.31 	 0.01
5HT2B 	 0.32 	 0.01
5HT2B 	 0.33 	 0.0
5HT2B 	 0.34 	 0.0
5HT2B 	 0.35000000000000003 	 0.0
5HT2B 	 0.36 	 0.0
5HT2B 	 0.37 	 0.0
5HT2B 	 0.3

receptor 	 p_cutoff 	 PPV
-----------------------------------
OPRD 	 0.0 	 0.02
OPRD 	 0.01 	 0.06
OPRD 	 0.02 	 0.07
OPRD 	 0.03 	 0.09
OPRD 	 0.04 	 0.1
OPRD 	 0.05 	 0.12
OPRD 	 0.06 	 0.14
OPRD 	 0.07 	 0.15
OPRD 	 0.08 	 0.16
OPRD 	 0.09 	 0.17
OPRD 	 0.1 	 0.18
OPRD 	 0.11 	 0.2
OPRD 	 0.12 	 0.2
OPRD 	 0.13 	 0.22
OPRD 	 0.14 	 0.25
OPRD 	 0.15 	 0.27
OPRD 	 0.16 	 0.29
OPRD 	 0.17 	 0.31
OPRD 	 0.18 	 0.31
OPRD 	 0.19 	 0.34
OPRD 	 0.2 	 0.37
OPRD 	 0.21 	 0.41
OPRD 	 0.22 	 0.45
OPRD 	 0.23 	 0.48
OPRD 	 0.24 	 0.55
OPRD 	 0.25 	 0.58
OPRD 	 0.26 	 0.61
OPRD 	 0.27 	 0.65
OPRD 	 0.28 	 0.66
OPRD 	 0.29 	 0.68
OPRD 	 0.3 	 0.7
OPRD 	 0.31 	 0.75
OPRD 	 0.32 	 0.8
OPRD 	 0.33 	 0.79
OPRD 	 0.34 	 0.86
OPRD 	 0.35000000000000003 	 0.94
OPRD 	 0.36 	 0.94
OPRD 	 0.37 	 0.94
OPRD 	 0.38 	 0.93
OPRD 	 0.39 	 0.93
OPRD 	 0.4 	 0.93
OPRD 	 0.41000000000000003 	 0.92
OPRD 	 0.42 	 0.91
OPRD 	 0.43 	 0.89
OPRD 	 0.44 	 0.89
OPRD 	 0.45 	 0.86
OPRD 	 0.46 	 0.86
OPRD 	 0.4700000000000000

In [11]:
nestedCV_pcutoffrange(2)

Predictors:  ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 's_score', 'hyd_prop', 'catdon_prop', 'aniacc_prop', 'quality']
receptor 	 p_cutoff 	 PPV
-----------------------------------
5HT2B 	 0.0 	 0.03
5HT2B 	 0.01 	 0.06
5HT2B 	 0.02 	 0.07
5HT2B 	 0.03 	 0.07
5HT2B 	 0.04 	 0.08
5HT2B 	 0.05 	 0.08
5HT2B 	 0.06 	 0.09
5HT2B 	 0.07 	 0.09
5HT2B 	 0.08 	 0.09
5HT2B 	 0.09 	 0.09
5HT2B 	 0.1 	 0.08
5HT2B 	 0.11 	 0.08
5HT2B 	 0.12 	 0.08
5HT2B 	 0.13 	 0.07
5HT2B 	 0.14 	 0.06
5HT2B 	 0.15 	 0.06
5HT2B 	 0.16 	 0.06
5HT2B 	 0.17 	 0.06
5HT2B 	 0.18 	 0.05
5HT2B 	 0.19 	 0.05
5HT2B 	 0.2 	 0.05
5HT2B 	 0.21 	 0.04
5HT2B 	 0.22 	 0.03
5HT2B 	 0.23 	 0.03
5HT2B 	 0.24 	 0.03
5HT2B 	 0.25 	 0.02
5HT2B 	 0.26 	 0.02
5HT2B 	 0.27 	 0.01
5HT2B 	 0.28 	 0.01
5HT2B 	 0.29 	 0.01
5HT2B 	 0.3 	 0.0
5HT2B 	 0.31 	 0.0
5HT2B 	 0.32 	 0.0
5HT2B 	 0.33 	 0.0
5HT2B 	 0.34 	 0.0
5HT2B 	 0.35000000000000003 	 0.0
5HT2B 	 0.36 	 0.0
5HT2B 	 0.37 	 0.0
5HT2B 	 0.38 	

OPRD 	 0.12 	 0.2
OPRD 	 0.13 	 0.22
OPRD 	 0.14 	 0.24
OPRD 	 0.15 	 0.27
OPRD 	 0.16 	 0.3
OPRD 	 0.17 	 0.33
OPRD 	 0.18 	 0.33
OPRD 	 0.19 	 0.36
OPRD 	 0.2 	 0.38
OPRD 	 0.21 	 0.41
OPRD 	 0.22 	 0.46
OPRD 	 0.23 	 0.51
OPRD 	 0.24 	 0.53
OPRD 	 0.25 	 0.57
OPRD 	 0.26 	 0.62
OPRD 	 0.27 	 0.67
OPRD 	 0.28 	 0.76
OPRD 	 0.29 	 0.79
OPRD 	 0.3 	 0.78
OPRD 	 0.31 	 0.81
OPRD 	 0.32 	 0.85
OPRD 	 0.33 	 0.85
OPRD 	 0.34 	 0.89
OPRD 	 0.35000000000000003 	 0.94
OPRD 	 0.36 	 0.93
OPRD 	 0.37 	 0.93
OPRD 	 0.38 	 0.92
OPRD 	 0.39 	 0.92
OPRD 	 0.4 	 0.92
OPRD 	 0.41000000000000003 	 0.9
OPRD 	 0.42 	 0.88
OPRD 	 0.43 	 0.86
OPRD 	 0.44 	 0.83
OPRD 	 0.45 	 0.83
OPRD 	 0.46 	 0.8
OPRD 	 0.47000000000000003 	 0.8
OPRD 	 0.48 	 0.5
OPRD 	 0.49 	 0.5
OPRD 	 0.5 	 0.0
OPRD 	 0.51 	 0.0
OPRD 	 0.52 	 0.0
OPRD 	 0.53 	 0.0
OPRD 	 0.54 	 0.0
OPRD 	 0.55 	 0.0
OPRD 	 0.56 	 0.0
OPRD 	 0.5700000000000001 	 0.0
OPRD 	 0.58 	 0.0
OPRD 	 0.59 	 0.0
OPRD 	 0.6 	 0.0
OPRD 	 0.61 	 0.0
OPRD 	 0.62 	 0

In [12]:
nestedCV_pcutoffrange(3)

Predictors:  ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 's_score', 'hyd_prop', 'catdon_prop', 'aniacc_prop', 'quality']
receptor 	 p_cutoff 	 PPV
-----------------------------------
5HT2B 	 0.0 	 0.03
5HT2B 	 0.01 	 0.06
5HT2B 	 0.02 	 0.07
5HT2B 	 0.03 	 0.08
5HT2B 	 0.04 	 0.08
5HT2B 	 0.05 	 0.09
5HT2B 	 0.06 	 0.09
5HT2B 	 0.07 	 0.09
5HT2B 	 0.08 	 0.09
5HT2B 	 0.09 	 0.09
5HT2B 	 0.1 	 0.09
5HT2B 	 0.11 	 0.08
5HT2B 	 0.12 	 0.08
5HT2B 	 0.13 	 0.07
5HT2B 	 0.14 	 0.07
5HT2B 	 0.15 	 0.07
5HT2B 	 0.16 	 0.06
5HT2B 	 0.17 	 0.06
5HT2B 	 0.18 	 0.06
5HT2B 	 0.19 	 0.05
5HT2B 	 0.2 	 0.05
5HT2B 	 0.21 	 0.05
5HT2B 	 0.22 	 0.04
5HT2B 	 0.23 	 0.04
5HT2B 	 0.24 	 0.03
5HT2B 	 0.25 	 0.03
5HT2B 	 0.26 	 0.03
5HT2B 	 0.27 	 0.03
5HT2B 	 0.28 	 0.03
5HT2B 	 0.29 	 0.02
5HT2B 	 0.3 	 0.02
5HT2B 	 0.31 	 0.02
5HT2B 	 0.32 	 0.01
5HT2B 	 0.33 	 0.01
5HT2B 	 0.34 	 0.01
5HT2B 	 0.35000000000000003 	 0.01
5HT2B 	 0.36 	 0.01
5HT2B 	 0.37 	 0.01
5HT2B 

M1 	 0.9500000000000001 	 0.0
M1 	 0.96 	 0.0
M1 	 0.97 	 0.0
M1 	 0.98 	 0.0
M1 	 0.99 	 0.0
receptor 	 p_cutoff 	 PPV
-----------------------------------
OPRD 	 0.0 	 0.02
OPRD 	 0.01 	 0.06
OPRD 	 0.02 	 0.07
OPRD 	 0.03 	 0.09
OPRD 	 0.04 	 0.1
OPRD 	 0.05 	 0.11
OPRD 	 0.06 	 0.13
OPRD 	 0.07 	 0.14
OPRD 	 0.08 	 0.15
OPRD 	 0.09 	 0.16
OPRD 	 0.1 	 0.17
OPRD 	 0.11 	 0.18
OPRD 	 0.12 	 0.18
OPRD 	 0.13 	 0.19
OPRD 	 0.14 	 0.2
OPRD 	 0.15 	 0.22
OPRD 	 0.16 	 0.25
OPRD 	 0.17 	 0.26
OPRD 	 0.18 	 0.29
OPRD 	 0.19 	 0.3
OPRD 	 0.2 	 0.32
OPRD 	 0.21 	 0.33
OPRD 	 0.22 	 0.34
OPRD 	 0.23 	 0.35
OPRD 	 0.24 	 0.39
OPRD 	 0.25 	 0.43
OPRD 	 0.26 	 0.47
OPRD 	 0.27 	 0.52
OPRD 	 0.28 	 0.56
OPRD 	 0.29 	 0.61
OPRD 	 0.3 	 0.65
OPRD 	 0.31 	 0.68
OPRD 	 0.32 	 0.71
OPRD 	 0.33 	 0.74
OPRD 	 0.34 	 0.72
OPRD 	 0.35000000000000003 	 0.74
OPRD 	 0.36 	 0.75
OPRD 	 0.37 	 0.77
OPRD 	 0.38 	 0.8
OPRD 	 0.39 	 0.86
OPRD 	 0.4 	 0.89
OPRD 	 0.41000000000000003 	 0.89
OPRD 	 0.42 	 0.94
OPRD 	

In [13]:
nestedCV_pcutoffrange(4)

Predictors:  ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 's_score', 'hyd_prop', 'catdon_prop', 'aniacc_prop', 'quality']
receptor 	 p_cutoff 	 PPV
-----------------------------------
5HT2B 	 0.0 	 0.03
5HT2B 	 0.01 	 0.07
5HT2B 	 0.02 	 0.08
5HT2B 	 0.03 	 0.09
5HT2B 	 0.04 	 0.09
5HT2B 	 0.05 	 0.08
5HT2B 	 0.06 	 0.08
5HT2B 	 0.07 	 0.07
5HT2B 	 0.08 	 0.06
5HT2B 	 0.09 	 0.05
5HT2B 	 0.1 	 0.06
5HT2B 	 0.11 	 0.05
5HT2B 	 0.12 	 0.05
5HT2B 	 0.13 	 0.04
5HT2B 	 0.14 	 0.04
5HT2B 	 0.15 	 0.04
5HT2B 	 0.16 	 0.03
5HT2B 	 0.17 	 0.03
5HT2B 	 0.18 	 0.03
5HT2B 	 0.19 	 0.03
5HT2B 	 0.2 	 0.03
5HT2B 	 0.21 	 0.03
5HT2B 	 0.22 	 0.03
5HT2B 	 0.23 	 0.03
5HT2B 	 0.24 	 0.02
5HT2B 	 0.25 	 0.02
5HT2B 	 0.26 	 0.02
5HT2B 	 0.27 	 0.02
5HT2B 	 0.28 	 0.01
5HT2B 	 0.29 	 0.0
5HT2B 	 0.3 	 0.0
5HT2B 	 0.31 	 0.0
5HT2B 	 0.32 	 0.0
5HT2B 	 0.33 	 0.0
5HT2B 	 0.34 	 0.0
5HT2B 	 0.35000000000000003 	 0.0
5HT2B 	 0.36 	 0.0
5HT2B 	 0.37 	 0.0
5HT2B 	 0.38 	 

H1 	 0.9500000000000001 	 0.0
H1 	 0.96 	 0.0
H1 	 0.97 	 0.0
H1 	 0.98 	 0.0
H1 	 0.99 	 0.0
receptor 	 p_cutoff 	 PPV
-----------------------------------
OPRD 	 0.0 	 0.02
OPRD 	 0.01 	 0.07
OPRD 	 0.02 	 0.1
OPRD 	 0.03 	 0.13
OPRD 	 0.04 	 0.15
OPRD 	 0.05 	 0.17
OPRD 	 0.06 	 0.18
OPRD 	 0.07 	 0.19
OPRD 	 0.08 	 0.21
OPRD 	 0.09 	 0.22
OPRD 	 0.1 	 0.24
OPRD 	 0.11 	 0.26
OPRD 	 0.12 	 0.27
OPRD 	 0.13 	 0.28
OPRD 	 0.14 	 0.3
OPRD 	 0.15 	 0.32
OPRD 	 0.16 	 0.35
OPRD 	 0.17 	 0.38
OPRD 	 0.18 	 0.4
OPRD 	 0.19 	 0.44
OPRD 	 0.2 	 0.48
OPRD 	 0.21 	 0.5
OPRD 	 0.22 	 0.51
OPRD 	 0.23 	 0.56
OPRD 	 0.24 	 0.52
OPRD 	 0.25 	 0.57
OPRD 	 0.26 	 0.61
OPRD 	 0.27 	 0.67
OPRD 	 0.28 	 0.73
OPRD 	 0.29 	 0.71
OPRD 	 0.3 	 0.7
OPRD 	 0.31 	 0.68
OPRD 	 0.32 	 0.8
OPRD 	 0.33 	 0.84
OPRD 	 0.34 	 0.88
OPRD 	 0.35000000000000003 	 0.93
OPRD 	 0.36 	 0.93
OPRD 	 0.37 	 0.92
OPRD 	 0.38 	 0.92
OPRD 	 0.39 	 0.92
OPRD 	 0.4 	 0.9
OPRD 	 0.41000000000000003 	 0.89
OPRD 	 0.42 	 0.89
OPRD 	 0.

In [14]:
nestedCV_pcutoffrange(5)

Predictors:  ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 's_score', 'hyd_prop', 'catdon_prop', 'aniacc_prop', 'quality']
receptor 	 p_cutoff 	 PPV
-----------------------------------
5HT2B 	 0.0 	 0.03
5HT2B 	 0.01 	 0.06
5HT2B 	 0.02 	 0.07
5HT2B 	 0.03 	 0.07
5HT2B 	 0.04 	 0.08
5HT2B 	 0.05 	 0.09
5HT2B 	 0.06 	 0.09
5HT2B 	 0.07 	 0.09
5HT2B 	 0.08 	 0.09
5HT2B 	 0.09 	 0.08
5HT2B 	 0.1 	 0.08
5HT2B 	 0.11 	 0.08
5HT2B 	 0.12 	 0.07
5HT2B 	 0.13 	 0.06
5HT2B 	 0.14 	 0.06
5HT2B 	 0.15 	 0.06
5HT2B 	 0.16 	 0.05
5HT2B 	 0.17 	 0.04
5HT2B 	 0.18 	 0.04
5HT2B 	 0.19 	 0.04
5HT2B 	 0.2 	 0.03
5HT2B 	 0.21 	 0.03
5HT2B 	 0.22 	 0.03
5HT2B 	 0.23 	 0.03
5HT2B 	 0.24 	 0.03
5HT2B 	 0.25 	 0.03
5HT2B 	 0.26 	 0.01
5HT2B 	 0.27 	 0.01
5HT2B 	 0.28 	 0.0
5HT2B 	 0.29 	 0.0
5HT2B 	 0.3 	 0.0
5HT2B 	 0.31 	 0.0
5HT2B 	 0.32 	 0.0
5HT2B 	 0.33 	 0.0
5HT2B 	 0.34 	 0.0
5HT2B 	 0.35000000000000003 	 0.0
5HT2B 	 0.36 	 0.0
5HT2B 	 0.37 	 0.0
5HT2B 	 0.38 	 0

H1 	 0.96 	 0.0
H1 	 0.97 	 0.0
H1 	 0.98 	 0.0
H1 	 0.99 	 0.0
receptor 	 p_cutoff 	 PPV
-----------------------------------
M1 	 0.0 	 0.07
M1 	 0.01 	 0.19
M1 	 0.02 	 0.24
M1 	 0.03 	 0.25
M1 	 0.04 	 0.25
M1 	 0.05 	 0.23
M1 	 0.06 	 0.22
M1 	 0.07 	 0.21
M1 	 0.08 	 0.2
M1 	 0.09 	 0.2
M1 	 0.1 	 0.21
M1 	 0.11 	 0.21
M1 	 0.12 	 0.23
M1 	 0.13 	 0.23
M1 	 0.14 	 0.23
M1 	 0.15 	 0.25
M1 	 0.16 	 0.25
M1 	 0.17 	 0.28
M1 	 0.18 	 0.31
M1 	 0.19 	 0.3
M1 	 0.2 	 0.31
M1 	 0.21 	 0.33
M1 	 0.22 	 0.25
M1 	 0.23 	 0.27
M1 	 0.24 	 0.26
M1 	 0.25 	 0.29
M1 	 0.26 	 0.27
M1 	 0.27 	 0.29
M1 	 0.28 	 0.31
M1 	 0.29 	 0.32
M1 	 0.3 	 0.35
M1 	 0.31 	 0.45
M1 	 0.32 	 0.47
M1 	 0.33 	 0.44
M1 	 0.34 	 0.5
M1 	 0.35000000000000003 	 0.55
M1 	 0.36 	 0.56
M1 	 0.37 	 0.56
M1 	 0.38 	 0.43
M1 	 0.39 	 0.43
M1 	 0.4 	 0.43
M1 	 0.41000000000000003 	 0.2
M1 	 0.42 	 0.2
M1 	 0.43 	 0.25
M1 	 0.44 	 0.33
M1 	 0.45 	 0.33
M1 	 0.46 	 0.33
M1 	 0.47000000000000003 	 0.33
M1 	 0.48 	 0.33
M1 	 0.

In [15]:
nestedCV_pcutoffrange(6)

Predictors:  ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 's_score', 'hyd_prop', 'catdon_prop', 'aniacc_prop', 'quality']
receptor 	 p_cutoff 	 PPV
-----------------------------------
5HT2B 	 0.0 	 0.03
5HT2B 	 0.01 	 0.06
5HT2B 	 0.02 	 0.07
5HT2B 	 0.03 	 0.07
5HT2B 	 0.04 	 0.08
5HT2B 	 0.05 	 0.09
5HT2B 	 0.06 	 0.09
5HT2B 	 0.07 	 0.09
5HT2B 	 0.08 	 0.09
5HT2B 	 0.09 	 0.09
5HT2B 	 0.1 	 0.08
5HT2B 	 0.11 	 0.08
5HT2B 	 0.12 	 0.07
5HT2B 	 0.13 	 0.07
5HT2B 	 0.14 	 0.06
5HT2B 	 0.15 	 0.06
5HT2B 	 0.16 	 0.05
5HT2B 	 0.17 	 0.05
5HT2B 	 0.18 	 0.05
5HT2B 	 0.19 	 0.04
5HT2B 	 0.2 	 0.04
5HT2B 	 0.21 	 0.03
5HT2B 	 0.22 	 0.04
5HT2B 	 0.23 	 0.03
5HT2B 	 0.24 	 0.03
5HT2B 	 0.25 	 0.03
5HT2B 	 0.26 	 0.03
5HT2B 	 0.27 	 0.03
5HT2B 	 0.28 	 0.01
5HT2B 	 0.29 	 0.0
5HT2B 	 0.3 	 0.0
5HT2B 	 0.31 	 0.0
5HT2B 	 0.32 	 0.0
5HT2B 	 0.33 	 0.0
5HT2B 	 0.34 	 0.0
5HT2B 	 0.35000000000000003 	 0.0
5HT2B 	 0.36 	 0.0
5HT2B 	 0.37 	 0.0
5HT2B 	 0.38 	 

H1 	 0.9500000000000001 	 0.0
H1 	 0.96 	 0.0
H1 	 0.97 	 0.0
H1 	 0.98 	 0.0
H1 	 0.99 	 0.0
receptor 	 p_cutoff 	 PPV
-----------------------------------
M1 	 0.0 	 0.07
M1 	 0.01 	 0.2
M1 	 0.02 	 0.24
M1 	 0.03 	 0.26
M1 	 0.04 	 0.25
M1 	 0.05 	 0.25
M1 	 0.06 	 0.23
M1 	 0.07 	 0.22
M1 	 0.08 	 0.21
M1 	 0.09 	 0.2
M1 	 0.1 	 0.21
M1 	 0.11 	 0.22
M1 	 0.12 	 0.22
M1 	 0.13 	 0.24
M1 	 0.14 	 0.23
M1 	 0.15 	 0.23
M1 	 0.16 	 0.25
M1 	 0.17 	 0.28
M1 	 0.18 	 0.29
M1 	 0.19 	 0.32
M1 	 0.2 	 0.35
M1 	 0.21 	 0.35
M1 	 0.22 	 0.31
M1 	 0.23 	 0.27
M1 	 0.24 	 0.25
M1 	 0.25 	 0.27
M1 	 0.26 	 0.28
M1 	 0.27 	 0.29
M1 	 0.28 	 0.3
M1 	 0.29 	 0.3
M1 	 0.3 	 0.34
M1 	 0.31 	 0.38
M1 	 0.32 	 0.5
M1 	 0.33 	 0.5
M1 	 0.34 	 0.47
M1 	 0.35000000000000003 	 0.5
M1 	 0.36 	 0.55
M1 	 0.37 	 0.6
M1 	 0.38 	 0.5
M1 	 0.39 	 0.43
M1 	 0.4 	 0.43
M1 	 0.41000000000000003 	 0.33
M1 	 0.42 	 0.25
M1 	 0.43 	 0.25
M1 	 0.44 	 0.33
M1 	 0.45 	 0.33
M1 	 0.46 	 0.33
M1 	 0.47000000000000003 	 0.

In [16]:
nestedCV_pcutoffrange(7)

Predictors:  ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 's_score', 'hyd_prop', 'catdon_prop', 'aniacc_prop', 'quality']
receptor 	 p_cutoff 	 PPV
-----------------------------------
5HT2B 	 0.0 	 0.03
5HT2B 	 0.01 	 0.06
5HT2B 	 0.02 	 0.07
5HT2B 	 0.03 	 0.07
5HT2B 	 0.04 	 0.08
5HT2B 	 0.05 	 0.09
5HT2B 	 0.06 	 0.09
5HT2B 	 0.07 	 0.09
5HT2B 	 0.08 	 0.09
5HT2B 	 0.09 	 0.08
5HT2B 	 0.1 	 0.07
5HT2B 	 0.11 	 0.07
5HT2B 	 0.12 	 0.07
5HT2B 	 0.13 	 0.06
5HT2B 	 0.14 	 0.05
5HT2B 	 0.15 	 0.05
5HT2B 	 0.16 	 0.04
5HT2B 	 0.17 	 0.04
5HT2B 	 0.18 	 0.04
5HT2B 	 0.19 	 0.04
5HT2B 	 0.2 	 0.04
5HT2B 	 0.21 	 0.03
5HT2B 	 0.22 	 0.03
5HT2B 	 0.23 	 0.03
5HT2B 	 0.24 	 0.03
5HT2B 	 0.25 	 0.03
5HT2B 	 0.26 	 0.03
5HT2B 	 0.27 	 0.03
5HT2B 	 0.28 	 0.02
5HT2B 	 0.29 	 0.01
5HT2B 	 0.3 	 0.0
5HT2B 	 0.31 	 0.0
5HT2B 	 0.32 	 0.0
5HT2B 	 0.33 	 0.0
5HT2B 	 0.34 	 0.0
5HT2B 	 0.35000000000000003 	 0.0
5HT2B 	 0.36 	 0.0
5HT2B 	 0.37 	 0.0
5HT2B 	 0.38 	

H1 	 0.97 	 0.0
H1 	 0.98 	 0.0
H1 	 0.99 	 0.0
receptor 	 p_cutoff 	 PPV
-----------------------------------
M1 	 0.0 	 0.07
M1 	 0.01 	 0.19
M1 	 0.02 	 0.23
M1 	 0.03 	 0.24
M1 	 0.04 	 0.22
M1 	 0.05 	 0.2
M1 	 0.06 	 0.17
M1 	 0.07 	 0.16
M1 	 0.08 	 0.17
M1 	 0.09 	 0.16
M1 	 0.1 	 0.17
M1 	 0.11 	 0.18
M1 	 0.12 	 0.2
M1 	 0.13 	 0.2
M1 	 0.14 	 0.21
M1 	 0.15 	 0.21
M1 	 0.16 	 0.23
M1 	 0.17 	 0.2
M1 	 0.18 	 0.24
M1 	 0.19 	 0.26
M1 	 0.2 	 0.28
M1 	 0.21 	 0.24
M1 	 0.22 	 0.25
M1 	 0.23 	 0.28
M1 	 0.24 	 0.26
M1 	 0.25 	 0.31
M1 	 0.26 	 0.32
M1 	 0.27 	 0.36
M1 	 0.28 	 0.41
M1 	 0.29 	 0.53
M1 	 0.3 	 0.56
M1 	 0.31 	 0.5
M1 	 0.32 	 0.5
M1 	 0.33 	 0.5
M1 	 0.34 	 0.43
M1 	 0.35000000000000003 	 0.33
M1 	 0.36 	 0.4
M1 	 0.37 	 0.4
M1 	 0.38 	 0.4
M1 	 0.39 	 0.5
M1 	 0.4 	 0.33
M1 	 0.41000000000000003 	 0.5
M1 	 0.42 	 0.5
M1 	 0.43 	 0.0
M1 	 0.44 	 0.0
M1 	 0.45 	 0.0
M1 	 0.46 	 0.0
M1 	 0.47000000000000003 	 0.0
M1 	 0.48 	 0.0
M1 	 0.49 	 0.0
M1 	 0.5 	 0.0
M1 	 

In [22]:
ph4_classify(0, 0.31)

Predicted  False  True 
Actual                 
0          14112     91
1            473      0 

PPV: 0.00


In [23]:
ph4_classify(1, 0.30)

Predicted  False  True 
Actual                 
0          14051     27
1            148      1 

PPV: 0.04


In [24]:
ph4_classify(2, 0.30)

Predicted  False  True 
Actual                 
0          14411     29
1            238      2 

PPV: 0.06


In [25]:
ph4_classify(3, 0.30)

Predicted  False  True 
Actual                 
0          14432    198
1            141     11 

PPV: 0.05


In [26]:
ph4_classify(4, 0.30)

Predicted  False  True 
Actual                 
0          13332     23
1            989     10 

PPV: 0.30


In [27]:
ph4_classify(5, 0.30)

Predicted  False  True 
Actual                 
0          13868      8
1            273     22 

PPV: 0.73


In [28]:
ph4_classify(6, 0.30)

Predicted  False  True 
Actual                 
0          14422     13
1            263     14 

PPV: 0.52


In [29]:
ph4_classify(7, 0.30)

Predicted  False  True 
Actual                 
0          13423    158
1            834    202 

PPV: 0.56
